In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/dataset.csv')

In [ ]:
target_col = "CTR"

if target_col in df.columns:

    y = df[target_col].astype(float)

    X = df.drop(columns=[target_col])

if {"Переходы", "Показы"}.issubset(df.columns):

    safe_shows = df["Показы"].replace(0, np.nan)

    y = (df["Переходы"] / safe_shows).fillna(0.0).astype(float)

    X = df.copy()

In [ ]:
leakage_cols = [c for c in ["CTR", "Переходы"] if c in X.columns]